In [80]:
import bitarray
import struct
import numpy as np

def print_hex_bytes(src):
    for i in range(8):
        str = src[i*8:(i+1)*8]
        print(' '.join('{:02X}'.format(a) for a in str))

In [81]:
R = [[14, 16],
     [52, 57],
     [23, 40],
     [5, 37],
     [25, 33],
     [46, 12],
     [58, 22],
     [32, 32]]

C240 = 0x1BD11BDAA9FC1A22
Nw = 4
Nr = 72
pi = [0,3,2,1]
block_length = 32

In [82]:
file = open("test.txt", "rb")
data_bytes = np.fromfile(file, dtype=np.uint8)
file.close()
print(data_bytes)

key = 'key of 32,64 or 128 bytes length'
tweak = 'tweak: 16 bytes '

key_bytes = np.frombuffer(bytearray(key, "utf-8"), dtype=np.uint8)
tweak_bytes = np.frombuffer(bytearray(tweak, "utf-8"), dtype=np.uint8)
print(len(data_bytes))

[ 49  50  51  52  53  54  55  56  57  48  13  10  48  57  56  55  54  53
  52  51  50  49  13  10 113 119 101 114 116 121 117 105 111 112  13  10
  97 115 100 102 103 104 106 107]
44


In [83]:
data_length = len(data_bytes)
print(np.dtype(data_bytes[0]))
print(len(data_bytes))
padding = block_length - (data_length % block_length)
if padding == 1:
    data_bytes = np.append(data_bytes, np.uint8(0x80))
elif padding > 1 and padding < block_length:
    data_bytes = np.append(data_bytes, np.uint8(0x80))
    for i in range(padding - 1):
        data_bytes = np.append(data_bytes, np.uint8(0x00))
print(np.dtype(data_bytes[0]))
print(len(data_bytes))


uint8
44
uint8
64


In [84]:
K = np.zeros((Nw+1), dtype=np.uint64)
for i in range(Nw):
    K[i] = np.frombuffer(key_bytes[i*8:i*(8)+8], np.uint64)

K[4] = C240
for i in range(Nw):
    K[4] ^= K[i]

K

C:\Users\Murav\AppData\Local\Temp\ipykernel_19100\4134362004.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  K[i] = np.frombuffer(key_bytes[i*8:i*(8)+8], np.uint64)


array([3684057122661295467, 2338053340664310834, 7310601557483467313,
       7526754601088589939,  397175047584170297], dtype=uint64)

In [85]:
T = np.zeros((3), dtype=np.uint64)
for i in range(2):
    T[i] = np.frombuffer(tweak_bytes[i*8:i*(8)+8], np.uint64)
print(T[0])
print(T[1])

T[2] = T[0] ^ T[1]
print(T[2])


3539893539983161204
2338324182462505014
1248446108863518530


C:\Users\Murav\AppData\Local\Temp\ipykernel_19100\2277352110.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  T[i] = np.frombuffer(tweak_bytes[i*8:i*(8)+8], np.uint64)


In [86]:
subKeys = np.zeros((Nr//4+1, Nw), np.uint64)
print(T)
print(K)

for round in range(np.uint64((Nr//4)+1)):
    for i in range(Nw):
        if i == 0:
            subKeys[round][i] = K[(round + i) % 5]
        elif i == 1:
            subKeys[round][i] = (K[(round + i) % 5] + T[round % 3])
        elif i == 2:
            subKeys[round][i] = (K[(round + i) % 5] + T[(round + 1) % 3])
        elif i == 3:
            kval = K[(round + i) % 5]
            value = kval + np.uint64(round)
            #print(type(value), type(kval), type(round))
            #print(f"K[({round} + {i}) % {5}] = {K[(round + i) % 5]}")
            #print(f"value = {value}")
            subKeys[round][i] = value
            #print(f"subKeys[{round}][{i}] = {subKeys[round][i]}")
            
    print(f"subKeys[{round}] = {subKeys[round]}")

[3539893539983161204 2338324182462505014 1248446108863518530]
[3684057122661295467 2338053340664310834 7310601557483467313
 7526754601088589939  397175047584170297]
subKeys[0] = [3684057122661295467 5877946880647472038 9648925739945972327
 7526754601088589939]
subKeys[1] = [2338053340664310834 9648925739945972327 8775200709952108469
  397175047584170298]
subKeys[2] = [7310601557483467313 8775200709952108469 3937068587567331501
 3684057122661295469]
subKeys[3] = [7526754601088589939 3937068587567331501 6022381305123800481
 2338053340664310837]
subKeys[4] = [ 397175047584170297 6022381305123800481 3586499449527829364
 7310601557483467317]
subKeys[5] = [ 3684057122661295467  3586499449527829364 10850495097466628517
  7526754601088589944]
subKeys[6] = [ 2338053340664310834 10850495097466628517  9865078783551094953
   397175047584170303]
subKeys[7] = [7310601557483467313 9865078783551094953 1645621156447688827
 3684057122661295474]
subKeys[8] = [7526754601088589939 1645621156447688827 72239

In [87]:
block_count = len(data_bytes) // (Nw * 8)
data_blocks = np.zeros((block_count, Nw), np.uint64)

for i in range(block_count):
    new_block = np.zeros((0), dtype=np.uint64)
    for word in range(Nw): 
        current_bytes = data_bytes[int(i*Nw + word*8): int(i*Nw+(word*8) + 8)]
        new_unit = np.frombuffer(current_bytes, np.uint64) 
        new_block = np.append(new_block, new_unit)
    data_blocks[i] = new_block


data_blocks

array([[4050765991979987505, 3978993149271683129,  724289206753703222,
        7599113487299999601],
       [ 724288137390929461, 3689632501694216496, 8243126020355207474,
         724358738889505140]], dtype=uint64)

In [88]:
def MIX(x0, x1, r):
    y0 = x0 + x1
    y1 = (x1 << np.uint64(r)) | (x1 >> np.uint64(64 - r))
    y1 ^= y0
    return (y0, y1)

def deMIX(x0, x1, r):
    x1 ^= x0
    y1 = (x1 << np.uint64(64 - r)) | (x1 >> np.uint64(r))
    y0 = x0 - y1
    return (y0, y1) 

In [89]:
encrypted_blocks = np.zeros((0), np.uint64)
for block in data_blocks:
    e = np.zeros_like(block)
    f = np.zeros_like(block)
    v = np.copy(block)
    
    for round in range(Nr):
        for i in range(Nw):
            if round % 4 == 0:
                e[i] = v[i] + subKeys[round//4][i]
                #print(f"subKeys[{round//4}][{i}] = {subKeys[round//4][i]}")
            else:
                e[i] = v[i]
        
        for i in range(Nw//2):
            f[i * 2], f[i * 2 + 1] = MIX(e[i*2], e[i*2 + 1], R[round % 8][i])
        for i in range(Nw):
            v[i] = f[pi[i]]

    for i in range(Nw):
        v[i] += subKeys[Nr//4][i]
    print(v)
    print(subKeys)
    encrypted_blocks = np.append(encrypted_blocks, v)

[11719035162925044900  5510683859213828704 10474525730632625250
 15059326532697117234]
[[ 3684057122661295467  5877946880647472038  9648925739945972327
   7526754601088589939]
 [ 2338053340664310834  9648925739945972327  8775200709952108469
    397175047584170298]
 [ 7310601557483467313  8775200709952108469  3937068587567331501
   3684057122661295469]
 [ 7526754601088589939  3937068587567331501  6022381305123800481
   2338053340664310837]
 [  397175047584170297  6022381305123800481  3586499449527829364
   7310601557483467317]
 [ 3684057122661295467  3586499449527829364 10850495097466628517
   7526754601088589944]
 [ 2338053340664310834 10850495097466628517  9865078783551094953
    397175047584170303]
 [ 7310601557483467313  9865078783551094953  1645621156447688827
   3684057122661295474]
 [ 7526754601088589939  1645621156447688827  7223950662644456671
   2338053340664310842]
 [  397175047584170297  7223950662644456671  4676377523126815848
   7310601557483467322]
 [ 3684057122661295467 

C:\Users\Murav\AppData\Local\Temp\ipykernel_19100\1684337085.py:2: RuntimeWarning: overflow encountered in scalar add
  y0 = x0 + x1
C:\Users\Murav\AppData\Local\Temp\ipykernel_19100\2225582889.py:10: RuntimeWarning: overflow encountered in scalar add
  e[i] = v[i] + subKeys[round//4][i]
C:\Users\Murav\AppData\Local\Temp\ipykernel_19100\2225582889.py:21: RuntimeWarning: overflow encountered in scalar add
  v[i] += subKeys[Nr//4][i]


In [90]:
print(encrypted_blocks)

[11719035162925044900  5510683859213828704 10474525730632625250
 15059326532697117234  1458758883764097602  4919327962687834854
 11710444152123123763 12647846953220826325]


In [91]:
decrypted_blocks = np.zeros_like(data_blocks, np.uint64)
print(encrypted_blocks)
for numb in range(len(encrypted_blocks)//Nw):
    block = encrypted_blocks[numb*Nw : (numb+1)*Nw]
    print(f"block = {block}")
    e = np.zeros_like(block)
    f = np.zeros_like(block)
    v = np.copy(block)

    for i in range(Nw):
        v[i] -= subKeys[Nr//4][i]
    
    for round in reversed(range(Nr)):
        for i in range(Nw):
            f[pi[i]] = v[i]
        
        for i in range(Nw//2):
            f0 = f[i * 2]
            f1 = f[i * 2 + 1]
            e[i*2], e[i*2 + 1] = deMIX(f0, f1, R[round%8][i])

        for i in range(Nw):
            if round % 4 == 0:
                v[i] = e[i] - subKeys[round//4][i]
            else:
                v[i] = e[i]

    #print(v)
    decrypted_blocks[numb] = v
decrypted_blocks

[11719035162925044900  5510683859213828704 10474525730632625250
 15059326532697117234  1458758883764097602  4919327962687834854
 11710444152123123763 12647846953220826325]
block = [11719035162925044900  5510683859213828704 10474525730632625250
 15059326532697117234]
block = [ 1458758883764097602  4919327962687834854 11710444152123123763
 12647846953220826325]


C:\Users\Murav\AppData\Local\Temp\ipykernel_19100\1684337085.py:10: RuntimeWarning: overflow encountered in scalar subtract
  y0 = x0 - y1
C:\Users\Murav\AppData\Local\Temp\ipykernel_19100\3999949010.py:24: RuntimeWarning: overflow encountered in scalar subtract
  v[i] = e[i] - subKeys[round//4][i]
C:\Users\Murav\AppData\Local\Temp\ipykernel_19100\3999949010.py:11: RuntimeWarning: overflow encountered in scalar subtract
  v[i] -= subKeys[Nr//4][i]


array([[4050765991979987505, 3978993149271683129,  724289206753703222,
        7599113487299999601],
       [ 724288137390929461, 3689632501694216496, 8243126020355207474,
         724358738889505140]], dtype=uint64)

In [92]:
print(f"source_blocks\n{data_blocks}")
print(f"encryped \n{encrypted_blocks}")
print(f"decrypted_blocks\n{decrypted_blocks}")

print(f"Input == output ? {np.array_equal(decrypted_blocks, data_blocks)}")

source_blocks
[[4050765991979987505 3978993149271683129  724289206753703222
  7599113487299999601]
 [ 724288137390929461 3689632501694216496 8243126020355207474
   724358738889505140]]
encryped 
[11719035162925044900  5510683859213828704 10474525730632625250
 15059326532697117234  1458758883764097602  4919327962687834854
 11710444152123123763 12647846953220826325]
decrypted_blocks
[[4050765991979987505 3978993149271683129  724289206753703222
  7599113487299999601]
 [ 724288137390929461 3689632501694216496 8243126020355207474
   724358738889505140]]
Input == output ? True
